# Configuration options

All Nengo Bones configuration is done through the `.nengobones.yml` file.
In this notebook we will demonstrate the different configuration options
available and how to use them to customize behaviour for different use cases.

First we'll define some utility functions we'll use throughout this notebook:

In [ ]:
def write_yml(contents):
    """Create a sample .nengo.yml file from a string."""

    nengo_yml = "pkg_name: example\nrepo_name: nengo-bones/example\n"
    nengo_yml += contents

    with open(".nengobones.yml", "w") as f:
        f.write(nengo_yml)


def display_contents(filename, sections=None):
    """
    Display the contents of a file.

    The 'sections' argument filters the file to show only the
    specified sections.
    """

    with open(filename) as f:
        data = f.readlines()

    # strip out blank lines
    data = [x for x in data if x.strip() != ""]

    if sections is None:
        display_data = data
    else:
        # pull out only the requested sections
        display_data = []
        for sec in sections:
            for i, line in enumerate(data):
                if ('"$COMMAND" == "%s"' % sec in line
                        or line.startswith(sec)):
                    section_data = data[i:]
                    break
            for i, line in enumerate(section_data[1:]):
                if not line.startswith(" "):
                    section_data = section_data[:i+1]
                    break
            display_data.extend(section_data)


    print("".join(display_data).rstrip("\n"))

## CI scripts

Nengo Bones uses a continuous integration (CI) setup
wherein build jobs are associated with different shell scripts,
and each script defines commands to be executed at different
stages during the build process.  These scripts are configured
through the ``ci_scripts`` section of ``.nengobones.yml``.
The basic step is to add an entry with `- template: scriptname`
for each script template that we want to be rendered.
As an example, in this section we will use the "test" template,
but all the options we describe here work for any of the templated scripts.

Normally the `.nengobones.yml` file is a text file sitting in the
root directory of the repository.  For demonstration purposes,
in this notebook we will be generating different config files on-the-fly
using the utility functions from above.

In [ ]:
nengobones_yml = """
ci_scripts:
    - template: test
"""
write_yml(nengobones_yml)  # create .nengobones.yml file
!generate-bones ci-scripts   # call the generate-bones script
display_contents("test.sh")  # display the contents of the generated file

There is a lot of information in that file that we don't
really need to worry about (that's the whole point of Nengo Bones,
to take care of those details for us).  We can see that the
overall structure is made up of behaviour defined for different
build stages (e.g. "install", "after_install", "before_script", etc.).
In most cases all the important action happens in the "install"
and "script" stages:

In [ ]:
display_contents("test.sh", sections=["install", "script"])

In the "install" stage we install any of the package requirements
needed to run the script. In this case we can see we're installing
`pytest`, `pytest-xdist`, and the current package (including any
optional dependencies defined in the `[tests]` `extra_requires`
directive).

If we need to add extra packages to this default installation,
that can be done with the `pip_install` or `conda_install` configuration
options (to install packages through `pip` or `conda`, respectively):

In [ ]:
nengobones_yml = """
ci_scripts:
    - template: test
      pip_install:
          - an-extra-pip-package
      conda_install:
          - an-extra-conda-package
"""
write_yml(nengobones_yml)
!generate-bones ci-scripts
display_contents("test.sh", sections=["install"])

Note that requirements should generally be added to the package
requirements defined in `setup.py` (in which case they would be
automatically installed in the `pip install -e ".[tests]"` step).
That way anyone can easily install the necessary packages and run
the tests without having to go through the CI scripts.
The `pip_install` and `conda_install` options are only necessary
if that isn't feasible for some reason.

The "script" stage is where the main work of the script is done.
In the case of the "test" script, this means calling `pytest` to 
run the test suite:

In [ ]:
display_contents("test.sh", sections=["script"])

Ignore the `$TEST_ARGS` part for now. We will cover that when
we come to the `.travis.yml` configuration.

As with the "install" stage, the "script" stage can also be
customized if we want to add extra commands, either before or
after the main script body:

In [ ]:
nengobones_yml = """
ci_scripts:
    - template: test
      pre_commands:
          - echo "this command will run at the beginning"
      post_commands:
          - echo "this command will run at the end"
"""
write_yml(nengobones_yml)
!generate-bones ci-scripts
display_contents("test.sh", sections=["script"])

We can also use the same template multiple times, passing
different options to generate different output scripts.
In this case we need to use the `output_name` config option
to distinguish different rendered scripts:

In [ ]:
nengobones_yml = """
ci_scripts:
    - template: test
      pre_commands:
          - echo "this is test"
    - template: test
      output_name: test2
      pre_commands:
          - echo "this is test2"
"""
write_yml(nengobones_yml)
!generate-bones ci-scripts
print("Contents of test.sh")
print("-------------------")
display_contents("test.sh", sections=["script"])
print("\nContents of test2.sh")
print("--------------------")
display_contents("test2.sh", sections=["script"])

### Test script options

The "test" script also has some configuration options specific
to that script.  First, we can collect coverage information while
tests are running by setting `coverage: true`:

In [ ]:
nengobones_yml = """
ci_scripts:
    - template: test
      coverage: true
"""
write_yml(nengobones_yml)
!generate-bones ci-scripts
display_contents("test.sh", sections=["install", "script", "after_script"])

Note that the script is now installing extra packages, adding
extra arguments to `pytest`, and uploading the results to
[Codecov](https://codecov.io/) at the end.

Nengo backends also often want to run the core Nengo tests
in addition to their own test suite.  This can be accomplished
by setting `nengo_tests: true`:

In [ ]:
nengobones_yml = """
ci_scripts:
    - template: test
      nengo_tests: true
"""
write_yml(nengobones_yml)
!generate-bones ci-scripts
display_contents("test.sh", sections=["script"])

## TravisCI config

As mentioned above, the general idea is that TravisCI will
create a number of build jobs, and each job will call one of
the scripts we generated.  TravisCI is configured through its
own `.travis.yml` file, so what we will be doing is using
`.nengobones.yml` to generate `.travis.yml`.  This is done
through the `travis_yml` section:

In [ ]:
nengobones_yml = """
travis_yml:
    jobs: []
"""
write_yml(nengobones_yml)
!generate-bones travis-yml
display_contents(".travis.yml")

As before, there is a lot of detail here that we don't need to
worry about because Nengo Bones will take care of it for us.
The important part to understand is the overall structure.

### Global variables

First, we create some global variables:

In [ ]:
display_contents(".travis.yml", sections=["env"])

This shows the default variables.  We can add arbitrary
variables to this list through the `global_vars` option:

In [ ]:
nengobones_yml = """
travis_yml:
    global_vars:
        VAR0: val0
        VAR1: val1
    jobs: []
"""
write_yml(nengobones_yml)
!generate-bones travis-yml
display_contents(".travis.yml", sections=["env"])

Note that these variables will be set in order, with later variables
overwriting earlier ones. So if, for example, we wanted to change the
default script, we could do that by setting:

In [ ]:
nengobones_yml = """
travis_yml:
    global_vars:
        SCRIPT: not-test
    jobs: []
"""
write_yml(nengobones_yml)
!generate-bones travis-yml
display_contents(".travis.yml", sections=["env"])

### Jobs

The next important part of the `.travis.yml` is the "jobs" section.

In [ ]:
display_contents(".travis.yml", sections=["jobs"])

It is currently empty because we haven't defined any jobs
in our `.nengobones.yml`.

This is done via the `jobs` section of `travis_yml`, which
specifies the builds we want to run during CI.  The most important
part of each job is specifying which script we want to run.
For example, we could create a single job that runs the "test" script:

In [ ]:
nengobones_yml = """
travis_yml:
    jobs:
        - script: test
"""
write_yml(nengobones_yml)
!generate-bones travis-yml
display_contents(".travis.yml", sections=["jobs"])

We can see that this causes the `$SCRIPT` variable to be set to `"test"`,
which will then cause `test.sh` to be run during the various build steps.

Similarly, we can set the `python_version` variable to change the Python
version being used for that job.  For example, we could create two jobs
that run the test suite with different python versions:

In [ ]:
nengobones_yml = """
travis_yml:
    jobs:
        - script: test
          python_version: 3.5
        - script: test
          python_version: 3.6
"""
write_yml(nengobones_yml)
!generate-bones travis-yml
display_contents(".travis.yml", sections=["jobs"])

The `test_args` option sets the `$TEST_ARGS` environment variable
that we saw in the `test.sh` script above.  Recall that this will
be passed to the `pytest` command, like `pytest my_project ... $TEST_ARGS`.
This can be useful for projects that define custom pytest arguments that
they might want to set for different jobs, e.g.:

In [ ]:
nengobones_yml = """
travis_yml:
    jobs:
        - script: test
          test_args: --do-a-special-test
"""
write_yml(nengobones_yml)
!generate-bones travis-yml
display_contents(".travis.yml", sections=["jobs"])

Note that all these options are simply shortcuts for setting
environment variables.  We can also set arbitrary environment
variables through the `env` option:

In [ ]:
nengobones_yml = """
travis_yml:
    jobs:
        - script: test
          env:
              VAR0: val0
"""
write_yml(nengobones_yml)
!generate-bones travis-yml
display_contents(".travis.yml", sections=["jobs"])

Any other options set for a job will be passed directly on to the
`.travis.yml` config for that job.  This opens up a wide range of
configuration options (see https://docs.travis-ci.com/ for more information).
As an example, we could divide jobs into different stages through
the `stage` option:

In [ ]:
nengobones_yml = """
travis_yml:
    jobs:
        - script: test
          stage: stage 0
        - script: docs
          stage: stage 1
"""
write_yml(nengobones_yml)
!generate-bones travis-yml
display_contents(".travis.yml", sections=["jobs"])

### Automatic deployment

Nengo Bones can also configure TravisCI to automatically deploy
releases to PyPI.  This is activated by setting the `pypi_user`
option, which should be the username for the PyPI account that will
be used to upload the releases.  You will also need to go into the
TravisCI settings for your repo and add a secure environment variable
named `PYPI_TOKEN` containing that account's password.  Finally, you
will need to make sure you are building the `deploy.sh` script by
adding `- template: deploy` to your `ci_scripts`.

In [ ]:
nengobones_yml = """
ci_scripts:
    - template: test
    - template: deploy

travis_yml:
    jobs:
        - script: test
    pypi_user: test_user
"""
write_yml(nengobones_yml)
!generate-bones travis-yml
display_contents(".travis.yml", sections=["jobs"])

This will trigger an automatic PyPI release for any tagged commit.
You can also do a test release by pushing to a branch named
`release-candidate-x.y.z` (where `x.y.z` is the version number you
want to test).

The only other setting related to the automatic deployment is
`deploy_dists`, which sets the distributions that will be built and
uploaded with each release.  We can see above that the default is an
`sdist`-only release, but we could, for example, add wheels by
modifying `deploy_dists`:

In [ ]:
nengobones_yml = """
ci_scripts:
    - template: test
    - template: deploy

travis_yml:
    jobs:
        - script: test
    pypi_user: test_user
    deploy_dists:
        - sdist
        - bdist_wheel
"""
write_yml(nengobones_yml)
!generate-bones travis-yml
display_contents(".travis.yml", sections=["jobs"])

### Python environment setup

The next section of the `.travis.yml` file sets up the python environment:

In [ ]:
display_contents(".travis.yml", sections=["before_install"])

Note that we are calling `generate-bones ... ci-scripts` in this
section in order to automatically generate all the scripts defined
in the `.nengobones.yml` `ci_scripts` discussed above.  We also call
`check-bones`, which will give an error if any of the templated files
are out of date.

There are no configuration options for this section (although note that
it is using the `$PYTHON_VERSION` variable we set in the global variables
above).

### Build steps

Finally we call our templated script in each of the build steps,
which will execute the behaviour we defined in those scripts.

In [ ]:
display_contents(".travis.yml", sections=[
    "install", "after_install", "before_script", "script"
])
print("...")

There are no configuration options for this section (instead this
behaviour is controlled by configuring the ci scripts, as described
in the `ci_scripts` section above).

## Codecov config

Nengo Bones will automatically upload coverage reports to
[Codecov](https://codecov.io/) if the `coverage: true` option
is set on the test script config.  Codecov reads the `.codecov.yml`
configuration file, which Nengo Bones also has a template for and
can be controlled through the `codecov_yml` option.

In [ ]:
nengobones_yml = """
codecov_yml: {}
"""
write_yml(nengobones_yml)
!generate-bones codecov-yml
display_contents(".codecov.yml")

The first thing to be noted is the `!ci.appveyor.com` line.
This tells Codecov not to wait for a coverage report from appveyor
(generally we assume that all the coverage is being checked on TravisCI).
This can be disabled by setting `skip_appveyor: false`:

In [ ]:
nengobones_yml = """
codecov_yml:
    skip_appveyor: false
"""
write_yml(nengobones_yml)
!generate-bones codecov-yml
display_contents(".codecov.yml", sections=["codecov"])

The other configuration options have to do with the target
coverage at either the project or patch level.  These determine 
whether the codecov PR status checks will be marked as passing
or failing.  The "project" coverage is the total lines of code
covered by tests for the whole project.  The default value is
"auto", meaning that the status check will pass if the total
coverage is >= the coverage in the base branch of the PR.
The "patch" coverage only looks at the lines of code modified
in the PR.  The default value is "100%", meaning that every
modified line of code needs to be covered by a test for the check to pass.

These can be modified by the `abs_target` and `diff_target`
config options, respectively:

In [ ]:
nengobones_yml = """
codecov_yml:
    abs_target: 80%
    diff_target: 90%
"""
write_yml(nengobones_yml)
!generate-bones codecov-yml
display_contents(".codecov.yml", sections=["coverage"])